In [1]:
import os
import re
import time
import random as rand
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [48]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/amelialui/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
<ipython-input-48-c026f0c15647>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
df = pd.read_csv ("Desktop/metis_shtuff/m4_dataacquisitionv15.csv")

In [4]:
#create list of champions
champions = []

In [49]:
https_start = "https://na.op.gg/leaderboards/tier?page=" #for to concatenate page number to end of this string
https_end = "&region=na"
driver.get(https_start+"1"+https_end)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6414 entries, 0 to 6413
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Username             6412 non-null   object
 1   ChampionToPredict    6404 non-null   object
 2   TierRankName         6414 non-null   object
 3   TierRank             6414 non-null   int64 
 4   LadderRank           6414 non-null   object
 5   TopChamp_S2022       6408 non-null   object
 6   TopChamp_Last20      6404 non-null   object
 7   AvgK_Last20          6414 non-null   object
 8   AvgD_Last20          6414 non-null   object
 9   AvgA_Last20          6414 non-null   object
 10  PrefPosition_Last20  6372 non-null   object
 11  LastPlayedChamp      6401 non-null   object
 12  LastPlayedResult     6414 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 651.5+ KB


In [56]:
df = df.replace(to_replace='None', value=-1).dropna()
df = df.dropna()
df = df.reset_index(drop=True)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6371 entries, 0 to 6370
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Username             6371 non-null   object
 1   ChampionToPredict    6371 non-null   object
 2   TierRankName         6371 non-null   object
 3   TierRank             6371 non-null   int64 
 4   LadderRank           6371 non-null   object
 5   TopChamp_S2022       6371 non-null   object
 6   TopChamp_Last20      6371 non-null   object
 7   AvgK_Last20          6371 non-null   object
 8   AvgD_Last20          6371 non-null   object
 9   AvgA_Last20          6371 non-null   object
 10  PrefPosition_Last20  6371 non-null   object
 11  LastPlayedChamp      6371 non-null   object
 12  LastPlayedResult     6371 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 647.2+ KB


In [ ]:
pagenumsscraped = []
for i in range(2,1010):
    string_pagenum = str(rand.randint(1,12972))
    while string_pagenum in pagenumsscraped:
        string_pagenum = str(rand.randint(1,12972))
    pagenumsscraped.append(string_pagenum)
    #getting ingredients to make soup
    driver.get(https_start+string_pagenum+https_end)
    #making soup
    globals()["soup"+string_pagenum] = bs(driver.page_source, 'html.parser')

    #make list of user urls on current leaderboard page opened
    names_list = globals()["soup"+string_pagenum].find_all(class_='css-1xdhyk6 ei93w700')
    numschosen = []
    urls = []
    for n in range(10):
        num = rand.randint(0,len(names_list)-1)
        while num in numschosen:
            num = rand.randint(0,len(names_list)-1)
        numschosen.append(num)
        div_info = str(names_list[num])
        href = div_info.split('''a href="''')[-1]
        href = href.split('''"><''')[0]
        url = "https://na.op.gg/" + href
        urls.append(url)

    #press on urls!
    for user_url in urls:
        driver.get(user_url)
        user_soup = bs(driver.page_source, 'html.parser')
        
        #now we collect data for dataframe, where each row is a user
        #get username
        try:
            if user_soup.find(class_="team") == None:
                username = user_soup.find(class_='name').text
            else:
                username = user_soup.find_all(class_='name')[1].text
        except:
            username = None
        
        #get the champion we want to predict
        try:
            championtopredict = user_soup.find_all(class_="css-ja2wlz e19epo2o3")[0].find(class_="name").text
        except:
            championtopredict = None
            
        #get tier rank
        try:
            tierrankname = user_soup.find(class_='tier-rank').text
            #convert tier into numerical value
            if tierrankname == "iron 4":
                tierrank = 1
            elif tierrankname == "iron 3":
                tierrank = 2
            elif tierrankname == "iron 2":
                tierrank = 3
            elif tierrankname == "iron 1":
                tierrank = 4
            elif tierrankname == "bronze 4":
                tierrank = 5
            elif tierrankname == "bronze 3":
                tierrank = 6
            elif tierrankname == "bronze 2":
                tierrank = 7
            elif tierrankname == "bronze 1":
                tierrank = 8
            elif tierrankname == "silver 4":
                tierrank = 9
            elif tierrankname == "silver 3":
                tierrank = 10
            elif tierrankname == "silver 2":
                tierrank = 11
            elif tierrankname == "silver 1":
                tierrank = 12
            elif tierrankname == "gold 4":
                tierrank = 13
            elif tierrankname == "gold 3":
                tierrank = 14
            elif tierrankname == "gold 2":
                tierrank = 15
            elif tierrankname == "gold 1":
                tierrank = 16
            elif tierrankname == "platinum 4":
                tierrank = 17
            elif tierrankname == "platinum 3":
                tierrank = 18
            elif tierrankname == "platinum 2":
                tierrank = 19
            elif tierrankname == "platinum 1":
                tierrank = 20
            elif tierrankname == "diamond 4":
                tierrank = 21
            elif tierrankname == "diamond 3":
                tierrank = 22
            elif tierrankname == "diamond 2":
                tierrank = 23
            elif tierrankname == "diamond 1":
                tierrank = 24
            elif tierrankname == "master":
                tierrank = 25
            elif tierrankname == "grandmaster":
                tierrank = 26
            elif tierrankname == "challenger":
                tierrank = 27 
        except:
            tierrank = -1
        
        #get ladder rank
        try:
            ladderrank = user_soup.find(class_="ranking").text.strip()
        except:
            ladderrank = -1
            
        #get top champion of choice of 2022
        try:
            topchamp_s2022 = user_soup.find(class_="champion-box").find(class_="name").text
            #add champion to champions list if not there already
            if topchamp_s2022 not in champions:
                champions.append(topchamp_s2022)
        except:
            topchamp_s2022 = None
            
        #get top champ of the last 20 games
        try:
            topchamp_last20 = user_soup.find(class_="most-champion").find(class_="name").text
        except:
            topchamp_last20 = None
            
        #get avg KDA per game of the last 20 games
        try:
            kda = user_soup.find(class_="k-d-a").text
        except:
            kda = "-1 / -1 / -1"
            
        #get average kills per game of the last 20 games
        #do less kills of the previous 20 games move the user to choose perhaps a higher damage champion?
        avgk_last20 = kda.split("/")[0].strip()
        
        #get average deaths per game of the last 20 games
        avgd_last20 = kda.split("/")[1].strip()
        
        #get average assists per game of the last 20 games
        avga_last20 = kda.split("/")[2].strip()
        
        #get preferred position of last 20 games
        try:
            prefposition_last20 = user_soup.find(class_="position-stats").find(class_="name").text
        except:
            prefposition_last20 = None
        
        #get last played champion
        try:
            lastplayedchamp = user_soup.find_all(class_="css-ja2wlz e19epo2o3")[1].find(class_="name").text
        except:
            lastplayedchamp = None
        
        #get last played result. was it a victory of defeat?
        try:
            winorloss = user_soup.find_all(class_="css-ja2wlz e19epo2o3")[1].find(class_="game-result").text
            if winorloss == "Victory":
                lastplayedresult = 1
            elif winorloss == "Defeat":
                lastplayedresult = 0
            else:
                lastplayedresult = -1
        except:
            winorloss = None
            lastplayedresult = -1
        
        #add new row, which represents a user, to dataframe    
        new_row = ({'Username': username,
                    'ChampionToPredict': championtopredict,
                    'TierRankName': tierrankname,
                    'TierRank': tierrank,
                    'LadderRank' : ladderrank,
                    'TopChamp_S2022': topchamp_s2022,
                    'TopChamp_Last20': topchamp_last20,
              
                    'AvgK_Last20': avgk_last20,
                    'AvgD_Last20' : avgd_last20, 
                    'AvgA_Last20': avga_last20,
                    'PrefPosition_Last20' : prefposition_last20,
                    'LastPlayedChamp' : lastplayedchamp,
                    'LastPlayedResult' : lastplayedresult})
        
        df = df.append(new_row, ignore_index = True)
                        
        time.sleep(4)
    
    #reset everything for next page
    urls = []
    time.sleep(4)

In [58]:
df.to_csv("Desktop/metis_shtuff/m4_dataacquisition21.csv")